In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/My\ paper

/content/drive/Shareddrives/My paper


In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

In [ ]:
member = pd.read_csv('dataset/91APP_MemberData.csv')

In [ ]:
order = pd.read_csv('dataset/91APP_OrderData.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## 處理行為資料

In [ ]:
# 十大事件
event_list = ['trafficSource', 'userRegisteration', 'mainPageView', 'productPageView', 'categoryPageView', 
              'activityPageView', 'search', 'addToCart', 'checkout', 'purchase']

bh_list = ['traffic', 'register', 'viewmainpage', 'viewproduct', 'viewcategory', 'viewactivity', 'search', 'add', 'checkout', 'purchase']

In [ ]:
months = range(1,13)
paths = []
for m in months:
  path = './Customer Value Prediction/TABLES/Behavior Data/Month '+ str(m) + '.csv'
  paths.append(path) 

behavior_data = [pd.read_csv(path, low_memory=False, encoding = "utf-8")[['did', 'uid', 'DateTime', 'bh']] for path in paths]

In [ ]:
label_data = pd.read_csv('./Customer Value Prediction/TABLES/Label Data.csv')

In [ ]:
label_data.head()

,MemberID,TotalSalesAmount,Active,VIP
0,RSQpIPxqSK2ooLPuUwWw8HhTN73mFGsmGPEmdarbdGM=,0.0,0.0,0.0
1,AqBrlkHltks4neA3Fmtq+yfiI4F7aZGODqKPeF3zGsA=,0.0,0.0,0.0
2,x/e5hrkgQCnCkhBzvk8Hs8i5dnX4tHRlrjHtKm6yvLg=,0.0,0.0,0.0
3,r/d/6e+NmTz+PFg45srs0HNMcUh1sWlRP27kZifm69I=,7612.0,1.0,1.0
4,wouxhlfcOpZcJH6lJhyuQ+4lV8b7X0foxY7YQiOQwC4=,0.0,0.0,0.0


In [ ]:
memberIDs = list(label_data['MemberID'].unique())
len(memberIDs)

83931

In [ ]:
member_data = member[member['MemberID'].isin(memberIDs)]
order_data = order[order['MemberID'].isin(memberIDs)]

In [ ]:
import collections
from datetime import date, timedelta, datetime
from sklearn.preprocessing import LabelEncoder

class DataPreprocessor:
  def __init__(self, member_data, order_data, behavior_data, memberIDs, predict_date, train_period):
    self.member_data = member_data
    self.order_data = order_data
    self.behavior_data = behavior_data # array: 每個位置為一個月的資料
    self.label_data = label_data
    self.train_period = train_period
    self.predict_start_date = datetime.strptime(predict_date, '%Y-%m-%d')
    self.feature_start_date = self.predict_start_date - timedelta(days=train_period)
    self.feature_end_date = self.predict_start_date - timedelta(days=1)
    print(self.feature_start_date, ',', self.feature_end_date,',', self.predict_start_date,)

    self.reset()

  def reset(self):
    self.memberIDs = list(self.label_data['MemberID'].unique())
    print('--------------Start to init member and order data--------------')
    self.preprocess_member_data()
    self.preprocess_order_data()
    print('--------------Finished: init member and order data--------------')

    # init data by processed member data
    self.complete_data = self.member_data
    self.feature_data= []
    self.feature_label_data = []

  def preprocess_member_data(self):
    print('Process member data')
    # extract register date
    member_date = self.member_data['RegisterDateTime'].str.split(' ', expand= True)
    member_date = member_date.rename(columns = {0: 'Date', 1: 'Time'}, inplace = False)
    self.member_data['Register_date'] = pd.to_datetime(member_date['Date'])

  def preprocess_order_data(self):
    print('Process order data')
    order_date = self.order_data['TradesDateTime'].str.split(' ', expand= True)
    order_date = order_date.rename(columns = {0: 'Date', 1: 'Time'}, inplace = False)
    self.order_data['Trade_date'] = pd.to_datetime(order_date['Date'], format = '%Y-%m-%d')
    self.order_data = self.order_data.sort_values(['TradesDateTime'])
    # only consider finish and return order
    self.order_data = self.order_data.loc[(self.order_data['Status'] == 'Finish') | (self.order_data['Status'] == 'Return')]
  
  def calculate_age(self, born, current):
    if born == '1900-01-01': # default age value
        return 0

    born = datetime.strptime(str(born).split(' ')[0], '%Y-%m-%d')
    return current.year - born.year - ((current.month, current.day) < (born.month, born.day))

  def add_age_col(self, df, date):
    df['Age'] = [0] * len(df)

    for index, row in df.iterrows():
        df.at[index, 'Age'] = self.calculate_age(row['Birthday'], date) 

  def calculate_lifespan(self, register, current):
    if register == '1900-01-01':
        return 10

    y = current.year - register.year - ((current.month, current.day) < (register.month, register.day))
    d = (current - register).days
  
    return [y, d]
  
  def add_lifespan_col(self, df, date):
    df['Lifespan'] = [0] * len(df)
    df['Lifespan_days'] = [0] * len(df)

    for index, row in df.iterrows():
        y, d = self.calculate_lifespan(row['Register_date'], date)
        df.at[index, 'Lifespan'] = y
        df.at[index, 'Lifespan_days'] = d

  def add_uid_col(self, df):
    df['uid'] = df['MemberID'].str.replace('+', '%2B')
    df['uid'] = df['uid'].str.replace('/', '%2F')
    df['uid'] = df['uid'].str.replace('=', '%3D')

  def get_session_data(self, uids):
    print('-------------------Start to process session times-------------------')
    id_dfs = []

    print('Get id table')
    for m in list(range(12)):
      df = behavior_data[m]
      id_dfs.append(df.loc[df['uid'].isin(uids)][['did', 'uid']].drop_duplicates(subset=['did'], keep='last'))
    id_table = pd.concat(id_dfs).drop_duplicates(subset=['did'], keep='last')
    
    print('Get behavior data of members')
    dfs = []

    for m in list(range(12)):
      df = behavior_data[m]
      dfs.append(df.loc[(df['did'].isin(id_table['did']))])
    
    behaviors = pd.concat(dfs)

    print('Process ids of member')
    # 一個 uid 有多個 did 的問題
    fill_uid = pd.merge(behaviors[['did']], id_table, how='left', on='did').rename(columns = {'uid': 'uid_filled'}, inplace = False)
    behaviors['uid_filled'] = list(fill_uid['uid_filled'])
    behaviors = behaviors.sort_values(by = ['uid_filled', 'DateTime'])   

    return self.get_session_sum(id_table, behaviors)

  def get_session_sum(self, id_table, df):
    print('Calculate session count')
    df_copy = df.copy()
    df_copy['Date'] = pd.to_datetime(df_copy['DateTime'], format='%Y-%m-%d %H:%M:%S').dt.date
    df_copy = df_copy.drop_duplicates(subset=['uid_filled', 'Date'], keep='first')
    result = pd.DataFrame(df_copy.groupby(by=['uid_filled']).size(), columns=['ActiveDays'])

    id_table_list = df.groupby('uid')['did'].apply(list).reset_index(name='did_list')
    result = pd.merge(id_table_list, result, left_on='uid', right_on='uid_filled', how='left')
    print('-------------------Finished: process session times-------------------')
    return result[['uid', 'ActiveDays']]

  # 更新: 只看哪幾天有 session ↑
  ## session 切換三種狀況的 code: https://colab.research.google.com/drive/1oTpPJxZ_GRdsqlib7KPgVo5LDd7QJd-W?usp=sharing

  def get_recency_data(self, date):
    finished_order = self.order_data.loc[(self.order_data['Status'] == 'Finish') & (self.order_data['Trade_date'] <= date)]
    finished_order = finished_order.drop_duplicates(subset=['MemberID'], keep='last') # only retain last order record of each member
    finished_order['today'] = [date]*len(finished_order)
    finished_order['Recency'] = (finished_order['today'] - finished_order['Trade_date']).dt.days + 1

    return(finished_order[['MemberID', 'Recency']])

  def get_freqency_data(self, start_date, end_date):
    order = self.order_data.loc[(self.order_data['Status'] == 'Finish') & (self.order_data['Trade_date'] >= start_date) & (self.order_data['Trade_date'] <= end_date)]
    order_count = pd.DataFrame(order.groupby(by=['MemberID']).size(), columns=['Frequency'])
    print(start_date, '-', end_date, 'has total ', len(order_count), 'members has order records.')

    return order_count

  def get_complete_order_data(self, end_date):
    order = self.order_data.loc[(self.order_data['Status'] == 'Finish') & (self.order_data['Trade_date'] <= end_date)]
    order_count = pd.DataFrame(order.groupby(by=['MemberID']).size(), columns=['CompleteOrders'])

    return order_count

  def get_promotion_data(self, end_date):
    order = self.order_data.loc[(self.order_data['Status'] == 'Finish') & (self.order_data['Trade_date'] <= end_date)]
    order_count = pd.DataFrame(order[order['TotalDiscount']!=0].groupby(by=['MemberID']).size(), columns=['PromotionUsed'])

    return order_count
  
  def get_return_order_data(self, end_date):
    order = self.order_data.loc[(self.order_data['Status'] == 'Return') & (self.order_data['Trade_date'] <= end_date)]
    order_count = pd.DataFrame(order.groupby(by=['MemberID']).size(), columns=['ReturnOrders'])

    return order_count
  
  def get_CAI_data(self, date):
    finished_order = self.order_data.loc[(self.order_data['Status'] == 'Finish') & (self.order_data['Trade_date'] <= date)]
    finished_order = finished_order.sort_values(['MemberID', 'TradesDateTime'])
    order = pd.merge(finished_order, self.member_data[['MemberID', 'Register_date']],on='MemberID', how='left')

    order['Weight'] = order.groupby(['MemberID']).cumcount()+1
    order['Shift_date'] = order['Trade_date'].shift(1)
    order['DateDiff'] = ((order['Trade_date'] - order['Shift_date']).dt.days) + 1 # 同一天也算一天
    order['DateDiff_Register'] = ((order['Trade_date'] - order['Register_date']).dt.days) + 1

    order.loc[order['Weight'] == 1, 'DateDiff'] = order['DateDiff_Register']
    order.loc[order['DateDiff'] < 0, 'DateDiff'] = 1 ## 修正 register date < trade date 的狀況

    order['DateDiff_w'] = (order['DateDiff'] * order['Weight']).astype(int)

    df_1 = pd.DataFrame(order.groupby(by=['MemberID']).sum().groupby(level=[0]).cumsum())
    df_2 = pd.DataFrame(order.groupby(by=['MemberID']).size(), columns=['count'])

    df_3 = pd.merge(df_1, df_2, on='MemberID')
    df_3['CAI'] = ((df_3['DateDiff'] / df_3['count']) - (df_3['DateDiff_w'] / df_3['Weight'])) / (df_3['DateDiff'] / df_3['count'])

    return pd.DataFrame(df_3['CAI'])

  def get_monetary_data(self, start_date, end_date):
    order = self.order_data.loc[(self.order_data['Trade_date'] >= start_date) & (self.order_data['Trade_date'] <= end_date)]
    monetary_data = order.groupby(by=['MemberID'])['TotalSalesAmount'].sum()

    finished_order = order.loc[(self.order_data['Status'] == 'Finish')]
    order_count = pd.DataFrame(finished_order.groupby(by=['MemberID']).size(), columns=['Frequency'])
    
    data = pd.merge(monetary_data, order_count, on='MemberID')
    data['Monetary'] = data['TotalSalesAmount'] / data['Frequency']

    return data[['Monetary']]


  def get_total_monetary_data(self, date):
    order = self.order_data.loc[self.order_data['Trade_date'] <= date]
    order_label= pd.DataFrame(order.groupby(by=['MemberID']).sum().groupby(level=[0]).cumsum()['TotalSalesAmount']).rename(columns={'TotalSalesAmount': 'Monetary_total'}) # get total net spending

    return order_label


  def get_data(self, df, start_date, end_date):
    print('-------------------Start to add some columns to member data------------------')
    self.add_age_col(df, end_date)
    self.add_lifespan_col(df, end_date)
    self.add_uid_col(df)
    print('------------------Finished: Add some columns to member data-------------------------')
    
    print('-------------------Start to add some columns to order data----------------------')
    order = self.get_freqency_data(start_date, end_date)
    data = pd.merge(df, order, on='MemberID', how='outer')

    recency_data = self.get_recency_data(end_date)
    data = pd.merge(data, recency_data, on='MemberID', how='left')

    complete_order_data = self.get_complete_order_data(end_date)
    data = pd.merge(data, complete_order_data, on='MemberID', how='left')

    promotion_data = self.get_promotion_data(end_date)
    data = pd.merge(data, promotion_data, on='MemberID', how='left')

    return_order_data = self.get_return_order_data(end_date)
    data = pd.merge(data, return_order_data, on='MemberID', how='left')
  
    CAI_data = self.get_CAI_data(end_date)
    data = pd.merge(data, CAI_data, on='MemberID', how='left')

    monetary_data = self.get_monetary_data(start_date, end_date)
    data = pd.merge(data, monetary_data, on='MemberID', how='left')

    total_monetary_data = self.get_total_monetary_data(end_date)
    data = pd.merge(data, total_monetary_data, on='MemberID', how='left')

    print('------------------Finished: Add some columns to order data-------------------------')
    uids = list(data['uid'])
    session_data = self.get_session_data(uids)
    print('Session data uid number: ', len(session_data['uid']), 'unique: ', len(session_data['uid'].unique()))
    data = pd.merge(data, session_data, on='uid', how='left')
 
    return data

  def preprocess_feature_data(self, df):
    columns = ['MemberID', 'Age', 'Lifespan', 'Lifespan_days', 'Recency', 
               'Frequency', 'Monetary', 'Monetary_total', 'CAI', 
               'ActiveDays', 'CompleteOrders', 'ReturnOrders', 'PromotionUsed'
          ]
    data = df[columns]
    
    recency_na = data['Recency'].max() + 1 # fill non recency
    
    data['Recency'] = data['Recency'].fillna(recency_na)
    data['CAI'] = data['CAI'].fillna(-1)
    data = data.fillna(0)
    
    data['PromotionUsedRate'] = data['PromotionUsed'] / data['CompleteOrders']
    data['PromotionUsedRate'] = data['PromotionUsedRate'].replace({ np.nan :  0, np.inf: 0})

    data['FinishedOrders'] = data['CompleteOrders'] - data['ReturnOrders']
    data['ReturnRate'] = data['ReturnOrders'] / data['CompleteOrders']
    data['ReturnRate'] = data['ReturnRate'].replace({ np.nan :  0, np.inf: 0})

    return data

  def merge_feature_label_data(self):
    data = pd.merge(self.feature_data, self.label_data, on='MemberID', how='left')

    return data

  def run(self):
    self.complete_data = self.get_data(self.complete_data, self.feature_start_date, self.feature_end_date)
    self.feature_data = self.preprocess_feature_data(self.complete_data)
    self.feature_label_data = self.merge_feature_label_data()

    self.feature_data = self.feature_label_data.drop(['TotalSalesAmount', 'Active', 'VIP'], axis=1)
    self.label_data = self.feature_label_data[['MemberID', 'TotalSalesAmount', 'Active', 'VIP']]

In [ ]:
data = DataPreprocessor(member_data, order_data, behavior_data, label_data, '2020-05-01', 366)
data.run()

2019-05-01 00:00:00 , 2020-04-30 00:00:00 , 2020-05-01 00:00:00
--------------Start to init member and order data--------------
Process member data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Process order data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


--------------Finished: init member and order data--------------
-------------------Start to add some columns to member data------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

------------------Finished: Add some columns to member data-------------------------
-------------------Start to add some columns to order data----------------------
2019-05-01 00:00:00 - 2020-04-30 00:00:00 has total  56040 members has order records.
------------------Finished: Add some columns to order data-------------------------
-------------------Start to process session times-------------------
Get id table
Get behavior data of members
Process ids of member
Calculate session count
-------------------Finished: process session times-------------------
Session data uid number:  85055 unique:  85055


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:250: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data.feature_label_data.loc[~data.feature_label_data['MemberID'].isin(list(data.label_data['MemberID']))]

,MemberID,Age,Lifespan,Lifespan_days,Recency,Frequency,Monetary,Monetary_total,CAI,ActiveDays,CompleteOrders,ReturnOrders,PromotionUsed,PromotionUsedRate,FinishedOrders,ReturnRate,TotalSalesAmount,Active,VIP


In [ ]:
data.feature_data.loc[~((data.feature_data['CAI']<=1)&(data.feature_data['CAI']>=-1))]

,MemberID,Age,Lifespan,Lifespan_days,Recency,Frequency,Monetary,Monetary_total,CAI,ActiveDays,CompleteOrders,ReturnOrders,PromotionUsed,PromotionUsedRate,FinishedOrders,ReturnRate


In [ ]:
data.label_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83931 entries, 0 to 83930
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MemberID          83931 non-null  object 
 1   TotalSalesAmount  83931 non-null  float64
 2   Active            83931 non-null  float64
 3   VIP               83931 non-null  float64
dtypes: float64(3), object(1)
memory usage: 3.2+ MB


In [ ]:
data.complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83931 entries, 0 to 83930
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   MemberID                  83931 non-null  object        
 1   RegisterSourceTypeDef     80792 non-null  object        
 2   RegisterDateTime          83931 non-null  object        
 3   Gender                    61750 non-null  object        
 4   Birthday                  83931 non-null  object        
 5   IsAppInstalled            83931 non-null  bool          
 6   IsEnableEmail             83931 non-null  bool          
 7   IsEnablePushNotification  83931 non-null  bool          
 8   IsEnableShortMessage      83931 non-null  bool          
 9   MemberCardLevel           83931 non-null  int64         
 10  Register_date             83931 non-null  datetime64[ns]
 11  Age                       83931 non-null  int64         
 12  Lifespan          

In [ ]:
data.feature_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83931 entries, 0 to 83930
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MemberID           83931 non-null  object 
 1   Age                83931 non-null  int64  
 2   Lifespan           83931 non-null  int64  
 3   Lifespan_days      83931 non-null  int64  
 4   Recency            83931 non-null  int64  
 5   Frequency          83931 non-null  float64
 6   Monetary           83931 non-null  float64
 7   Monetary_total     83931 non-null  float64
 8   CAI                83931 non-null  float64
 9   ActiveDays         83931 non-null  float64
 10  CompleteOrders     83931 non-null  int64  
 11  ReturnOrders       83931 non-null  float64
 12  PromotionUsed      83931 non-null  float64
 13  PromotionUsedRate  83931 non-null  float64
 14  FinishedOrders     83931 non-null  float64
 15  ReturnRate         83931 non-null  float64
dtypes: float64(10), int64(

In [ ]:
data.feature_label_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83931 entries, 0 to 83930
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MemberID           83931 non-null  object 
 1   Age                83931 non-null  int64  
 2   Lifespan           83931 non-null  int64  
 3   Lifespan_days      83931 non-null  int64  
 4   Recency            83931 non-null  int64  
 5   Frequency          83931 non-null  float64
 6   Monetary           83931 non-null  float64
 7   Monetary_total     83931 non-null  float64
 8   CAI                83931 non-null  float64
 9   ActiveDays         83931 non-null  float64
 10  CompleteOrders     83931 non-null  int64  
 11  ReturnOrders       83931 non-null  float64
 12  PromotionUsed      83931 non-null  float64
 13  PromotionUsedRate  83931 non-null  float64
 14  FinishedOrders     83931 non-null  float64
 15  ReturnRate         83931 non-null  float64
 16  TotalSalesAmount   839

In [ ]:
data.feature_label_data.head()

,MemberID,Age,Lifespan,Lifespan_days,Recency,Frequency,Monetary,Monetary_total,CAI,ActiveDays,CompleteOrders,ReturnOrders,PromotionUsed,PromotionUsedRate,FinishedOrders,ReturnRate,TotalSalesAmount,Active,VIP
0,sq6A9cXpkm6osmubTCLs8bWeANzqL5qGj9eHJszdQSU=,26,4,1517,123,2.0,840.00,5820.0,0.138333,16.0,5,5.0,1.0,0.200000,0.0,1.000000,0.0,0.0,0.0
1,5zctnMgeBBnl88Ncab+TgtLtTv4IKqEqHZOVDdMbzp0=,31,3,1225,8,4.0,2212.25,31367.0,0.090855,50.0,13,0.0,12.0,0.923077,13.0,0.000000,3635.0,1.0,1.0
2,CFwUYZ8qzgF+9DlCOT+VyjhQFqB3VoHz/83rwJoodFY=,30,3,1414,1035,0.0,0.00,10980.0,-0.186684,3.0,3,2.0,0.0,0.000000,1.0,0.666667,0.0,0.0,0.0
3,Sln7tpbmaTY2lN9m/PFIjHMFr8oWt2x5BpTAoQ5wV8I=,24,3,1427,271,2.0,703.00,11979.0,-0.059084,2.0,6,5.0,2.0,0.333333,1.0,0.833333,0.0,0.0,0.0
4,MJYoc2LF9yz2q+mJ7IlBbEf/oT5TvTP5s0Tu8t1pNMY=,27,4,1520,244,1.0,1016.00,10165.0,-0.318323,8.0,11,4.0,4.0,0.363636,7.0,0.363636,4840.0,1.0,1.0


In [ ]:
data.feature_label_data.groupby(by=['VIP']).size()

VIP
0.0    72733
1.0    11198
dtype: int64

In [ ]:
# data.complete_data.to_csv('feature_data(complete).csv', index=False, encoding = 'utf-8')
# data.feature_data.to_csv('feature_data.csv', index=False, encoding = 'utf-8')
# data.label_data.to_csv('label_data.csv', index=False, encoding = 'utf-8')
data.feature_label_data.to_csv('Handcrafted Features.csv', index=False, encoding = 'utf-8')

## 變數關係

In [ ]:
data.feature_label_data.columns

Index(['MemberID', 'Age', 'Lifespan', 'Lifespan_days', 'Recency', 'Frequency',
       'Monetary', 'Monetary_total', 'CAI', 'ActiveDays', 'CompleteOrders',
       'ReturnOrders', 'PromotionUsed', 'PromotionUsedRate', 'FinishedOrders',
       'ReturnRate', 'PurchaseRatio', 'TotalSalesAmount', 'Active', 'VIP'],
      dtype='object')

In [ ]:
X_cols = ['Age', 'Lifespan', 'Lifespan_days', 'Recency', 'Frequency',
       'Monetary', 'Monetary_total', 'CAI', 'ActiveDays', 'CompleteOrders',
       'ReturnOrders', 'PromotionUsed', 'PromotionUsedRate', 'FinishedOrders',
       'ReturnRate']

In [ ]:
X = data.feature_label_data[X_cols]
y = data.feature_label_data['VIP']

In [ ]:
import statsmodels.api as sm
X_cons = sm.add_constant(X.astype(float))
model = sm.OLS(y, X_cons)
model_fit = model.fit()
print(model_fit.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


                            OLS Regression Results                            
Dep. Variable:                    VIP   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.136
Method:                 Least Squares   F-statistic:                     881.7
Date:                Thu, 07 Apr 2022   Prob (F-statistic):               0.00
Time:                        07:55:18   Log-Likelihood:                -22412.
No. Observations:               83931   AIC:                         4.486e+04
Df Residuals:                   83915   BIC:                         4.501e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0091      0.00

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
